<a href="https://colab.research.google.com/github/danielapavas/ProgramacionLineal/blob/main/Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DANIELA ANDREA PAVAS BEDOYA
# YULY YESENIA ALVEAR ROMO

In [10]:
pip install pulp

In [11]:
import pulp
import re

In [12]:
#Solicitar el número de variables
num_variables = int(input("Ingrese el número de variables: "))

Ingrese el número de variables: 2


In [13]:
# Crear variables de decisión
variables = [pulp.LpVariable(f'x{i}', lowBound=0) for i in range(num_variables)]

In [14]:
#  Solicitar  la función objetivo
print("Ingrese la función objetivo en términos de las variables. Por ejemplo, '2*x0 + 3*x1':")
objetivo_str = input("Función objetivo: ")
terms = re.split(r'\s*\+\s*', objetivo_str)
objetivo = pulp.LpAffineExpression()
for term in terms:
    coef, var = term.split('*')
    coef = float(coef)
    if any(v.name == var for v in variables):
        variable = next(v for v in variables if v.name == var)
        objetivo += coef * variable
    else:
        print(f"La variable '{var}' no coincide con las variables definidas.")

# Imprimir la expresión lineal objetivo
print("Función objetivo:", objetivo)


Ingrese la función objetivo en términos de las variables. Por ejemplo, '2*x0 + 3*x1':
Función objetivo: 3*x0 + 5*x1
Función objetivo: 3.0*x0 + 5.0*x1


In [20]:

num_restricciones = int(input("Ingrese el número de restricciones: "))

var_dict = {v.name: v for v in variables}

restricciones = []
for i in range(num_restricciones):
    restriccion_str = input(f"Ingrese la restricción {i + 1} en términos de las variables (por ejemplo, 'x0+x1 <= 5'): ")

    restriccion_str = re.sub(r'x\d+', lambda x: f'var_dict["{x.group(0)}"]', restriccion_str)

    # Analizar la restricción
    restriccion_parts = re.split(r'\s+', restriccion_str)

    if len(restriccion_parts) != 3:
        print(f"La restricción '{restriccion_str}' no está en el formato correcto.")
    else:
        var1, op, rhs = restriccion_parts

        if op == "<=":
            sense = pulp.LpConstraintLE
        elif op == ">=":
            sense = pulp.LpConstraintGE
        elif op == "=":
            sense = pulp.LpConstraintEQ
        else:
            print(f"Operador '{op}' no válido en la restricción.")
            continue

        restriccion = pulp.LpConstraint(
            e=eval(restriccion_str),
            sense=sense,
            rhs=float(rhs),
            name=f'Restricción_{i}'
        )

        restricciones.append(restriccion)

Ingrese el número de restricciones: 3
Ingrese la restricción 1 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): x0 <= 4
Ingrese la restricción 2 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): 2*x1 <= 12
Ingrese la restricción 3 en términos de las variables (por ejemplo, 'x0+x1 <= 5'): 3*x0+2*x1 <= 18


In [21]:
problema = pulp.LpProblem("Problema", pulp.LpMaximize)  # O maximizar según su caso
problema.setObjective(objetivo)

for restriccion in restricciones:
    problema += restriccion

In [22]:
problema.solve()

1

In [23]:
print("Estado:", pulp.LpStatus[problema.status])
print("Valor óptimo de la función objetivo =", pulp.value(problema.objective))
for var in variables:
    print(f"Valor óptimo de {var.name} =", var.varValue)


Estado: Optimal
Valor óptimo de la función objetivo = 72.0
Valor óptimo de x0 = 4.0
Valor óptimo de x1 = 12.0
